<div align="center">

# Run-rate & Seasonality
Are we trending up across weeks into December? Any dips worth investigating?
</div>

In [ ]:

# TODO : 
# 1. Gets Dates Leading to December (Week 45-51)
# 2. Find Dips based on Revenue change per week of revenue

import pandas as pd
import numpy as np

In [59]:
df = pd.read_csv("./Sales-Data-Analysis.csv")

In [60]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce", dayfirst=True)
df["Date"]

0     2022-11-07
1     2022-11-07
2     2022-11-07
3     2022-11-08
4     2022-11-08
         ...    
249   2022-12-28
250   2022-12-29
251   2022-12-29
252   2022-12-29
253   2022-12-29
Name: Date, Length: 254, dtype: datetime64[ns]

In [61]:
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")
df["Quantity"] = pd.to_numeric(df["Quantity"], errors="coerce")
df["Revenue"] = df["Price"] * df["Quantity"]
df['Revenue']

0      2000.0143
1      2199.9920
2       999.9960
3      7400.0133
4      2000.0495
         ...    
249     999.9960
250    9800.0457
251    2800.0295
252    2199.9913
253    2000.0115
Name: Revenue, Length: 254, dtype: float64

In [62]:
df = df.dropna(subset=["Date", "Revenue"])
df

,Order ID,Date,Product,Price,Quantity,Purchase Type,Payment Method,Manager,City,Revenue
0,10452,2022-11-07,Fries,3.49,573.07,Online,Gift Card,Tom Jackson,London,2000.0143
1,10453,2022-11-07,Beverages,2.95,745.76,Online,Gift Card,Pablo Perez,Madrid,2199.9920
2,10454,2022-11-07,Sides & Other,4.99,200.40,In-store,Gift Card,Joao Silva,Lisbon,999.9960
3,10455,2022-11-08,Burgers,12.99,569.67,In-store,Credit Card,Walter Muller,Berlin,7400.0133
4,10456,2022-11-08,Chicken Sandwiches,9.95,201.01,In-store,Credit Card,Walter Muller,Berlin,2000.0495
...,...,...,...,...,...,...,...,...,...,...
249,10709,2022-12-28,Sides & Other,4.99,200.40,Drive-thru,Gift Card,Walter Muller,Berlin,999.9960
250,10710,2022-12-29,Burgers,12.99,754.43,Drive-thru,Gift Card,Walter Muller,Berlin,9800.0457
251,10711,2022-12-29,Chicken Sandwiches,9.95,281.41,Drive-thru,Gift Card,Walter Muller,Berlin,2800.0295
252,10712,2022-12-29,Fries,3.49,630.37,Drive-thru,Gift Card,Walter Muller,Berlin,2199.9913


In [63]:
df["Week"] = df["Date"].dt.isocalendar().week.astype(int)
df["Month"] = df["Date"].dt.month

In [64]:
weekly_rev = df.groupby("Week", as_index=False)["Revenue"].sum().sort_values("Week")
weekly_rev

,Week,Revenue
0,45,101074.6769
1,46,95639.6364
2,47,95000.1629
3,48,90800.5043
4,49,100600.5254
5,50,104200.3715
6,51,113799.8870
7,52,68400.0948


In [65]:
weekly_rev["WoW_Change_%"] = np.round(
    weekly_rev["Revenue"].pct_change(fill_method=None) * 100, 2
)
weekly_rev["RunRate_Avg"] = np.round(
    weekly_rev["Revenue"].expanding().mean(), 2
)

In [66]:
dips = weekly_rev[weekly_rev["WoW_Change_%"] < 0]

print(weekly_rev.head(10))
print("\n")
print(dips)


   Week      Revenue  WoW_Change_%  RunRate_Avg
0    45  101074.6769           NaN    101074.68
1    46   95639.6364         -5.38     98357.16
2    47   95000.1629         -0.67     97238.16
3    48   90800.5043         -4.42     95628.75
4    49  100600.5254         10.79     96623.10
5    50  104200.3715          3.58     97885.98
6    51  113799.8870          9.21    100159.39
7    52   68400.0948        -39.89     96189.48


   Week     Revenue  WoW_Change_%  RunRate_Avg
1    46  95639.6364         -5.38     98357.16
2    47  95000.1629         -0.67     97238.16
3    48  90800.5043         -4.42     95628.75
7    52  68400.0948        -39.89     96189.48
